Understanding sales trends is crucial for any business striving to maximize its performance and cater to customer needs effectively. One of the most valuable insights comes from analyzing how sales vary across different categories and days of the week.

In this blog, we will walk through a practical example of generating a sales report from raw data using SQL. This report will break down sales by product categories, such as books, phones, and glasses, and map them to the specific day of the week they were ordered. By doing so, we aim to uncover patterns in purchasing behavior that can help businesses make informed decisions, such as optimizing inventory, launching day-specific promotions, or identifying underperforming product lines.

#### Table: Orders
| Column Name  | Type    |
|--------------|---------|
| order_id     | int     |
| customer_id  | int     |
| order_date   | date    |
| item_id      | varchar |
| quantity     | int     |

- (order_id, item_id) is the primary key for this table.
- This table contains information about orders placed, where `order_date` is the date `item_id` was ordered by the customer with `customer_id`.

#### Table: Items
| Column Name   | Type    |
|---------------|---------|
| item_id       | varchar |
| item_name     | varchar |
| item_category | varchar |

- `item_id` is the primary key for this table.
- This table contains details about items, including their name and category.

---

### Problem Statement

As a business owner, you would like to generate a sales report that summarizes how many units in each category were ordered on each day of the week.

#### Expected Output

The output should display the number of units sold for each item category, organized by the day of the week. The result table should be sorted by category.

##### Input:

**Orders Table:**
| order_id | customer_id | order_date | item_id | quantity |
|----------|-------------|------------|---------|----------|
| 1        | 1           | 2020-06-01 | 1       | 10       |
| 2        | 1           | 2020-06-08 | 2       | 10       |
| 3        | 2           | 2020-06-02 | 1       | 5        |
| 4        | 3           | 2020-06-03 | 3       | 5        |
| 5        | 4           | 2020-06-04 | 4       | 1        |
| 6        | 4           | 2020-06-05 | 5       | 5        |
| 7        | 5           | 2020-06-05 | 1       | 10       |
| 8        | 5           | 2020-06-14 | 4       | 5        |
| 9        | 5           | 2020-06-21 | 3       | 5        |

**Items Table:**
| item_id | item_name       | item_category |
|---------|-----------------|---------------|
| 1       | LC Alg. Book    | Book          |
| 2       | LC DB. Book     | Book          |
| 3       | LC Smartphone   | Phone         |
| 4       | LC Phone 2020   | Phone         |
| 5       | LC SmartGlass   | Glasses       |
| 6       | LC T-Shirt XL   | T-Shirt       |

##### Output:

| Category | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
|----------|--------|---------|-----------|----------|--------|----------|--------|
| Book     | 20     | 5       | 0         | 0        | 10     | 0        | 0      |
| Glasses  | 0      | 0       | 0         | 0        | 5      | 0        | 0      |
| Phone    | 0      | 0       | 5         | 1        | 0      | 0        | 10     |
| T-Shirt  | 0      | 0       | 0         | 0        | 0      | 0        | 0      |

---

#### Explanation:

1. **Monday**:
   - On 2020-06-01 and 2020-06-08, 20 units of category **Book** (10 + 10) were sold.

2. **Tuesday**:
   - On 2020-06-02, 5 units of category **Book** were sold.

3. **Wednesday**:
   - On 2020-06-03, 5 units of category **Phone** were sold.

4. **Thursday**:
   - On 2020-06-04, 1 unit of category **Phone** was sold.

5. **Friday**:
   - On 2020-06-05, 10 units of category **Book** and 5 units of category **Glasses** were sold.

6. **Sunday**:
   - On 2020-06-14 and 2020-06-21, 10 units of category **Phone** (5 + 5) were sold.

7. No sales occurred for the **T-Shirt** category.


In [24]:
import pandas as pd

data = [[1, 1, '2020-06-01', 1, 10], 
        [2, 1, '2020-06-08', 2, 10], 
        [3, 2, '2020-06-02', 1, 5], 
        [4, 3, '2020-06-03', 3, 5], 
        [5, 4, '2020-06-04', 4, 1], 
        [6, 4, '2020-06-05', 5, 5], 
        [7, 5, '2020-06-05', 1, 10], 
        [8, 5, '2020-06-14', 4, 5], 
        [9, 5, '2020-06-21', 3, 5]]
orders = pd.DataFrame(data, 
          columns=['order_id', 
                   'customer_id', 
                   'order_date', 
                   'item_id', 
                   'quantity']).astype({'order_id':'Int64', 
                   'customer_id':'Int64', 
                   'order_date':'datetime64[ns]', 
                   'item_id':'object', 'quantity':'Int64'})
display(orders)

,order_id,customer_id,order_date,item_id,quantity
0,1,1,2020-06-01,1,10
1,2,1,2020-06-08,2,10
2,3,2,2020-06-02,1,5
3,4,3,2020-06-03,3,5
4,5,4,2020-06-04,4,1
5,6,4,2020-06-05,5,5
6,7,5,2020-06-05,1,10
7,8,5,2020-06-14,4,5
8,9,5,2020-06-21,3,5


In [25]:
data = [[1, 'LC Alg. Book', 'Book'], 
        [2, 'LC DB. Book', 'Book'], 
        [3, 'LC SmarthPhone', 'Phone'], 
        [4, 'LC Phone 2020', 'Phone'], 
        [5, 'LC SmartGlass', 'Glasses'], 
        [6, 'LC T-Shirt XL', 'T-shirt']]
items = pd.DataFrame(data, 
        columns=['item_id', 
                 'item_name', 
                 'item_category']).astype({'item_id':'object', 
                 'item_name':'object', 
                 'item_category':'object'})
display(items)

,item_id,item_name,item_category
0,1,LC Alg. Book,Book
1,2,LC DB. Book,Book
2,3,LC SmarthPhone,Phone
3,4,LC Phone 2020,Phone
4,5,LC SmartGlass,Glasses
5,6,LC T-Shirt XL,T-shirt


**Step 1. Merge Orders with Items**
- This line merges the orders DataFrame with the items DataFrame using a left join. Each row in the orders table will now include information about the corresponding item from the items table, such as its name and category. This is necessary to associate the item category with each order for further analysis.

In [26]:
orders = orders.merge(items, how="left")
display(orders)

,order_id,customer_id,order_date,item_id,quantity,item_name,item_category
0,1,1,2020-06-01,1,10,LC Alg. Book,Book
1,2,1,2020-06-08,2,10,LC DB. Book,Book
2,3,2,2020-06-02,1,5,LC Alg. Book,Book
3,4,3,2020-06-03,3,5,LC SmarthPhone,Phone
4,5,4,2020-06-04,4,1,LC Phone 2020,Phone
5,6,4,2020-06-05,5,5,LC SmartGlass,Glasses
6,7,5,2020-06-05,1,10,LC Alg. Book,Book
7,8,5,2020-06-14,4,5,LC Phone 2020,Phone
8,9,5,2020-06-21,3,5,LC SmarthPhone,Phone


**Step 2. Extract Day Name from Order Date**
- The order_date column is converted into the day name (e.g., "Monday", "Tuesday"). A new column called day is added to the orders DataFrame to store this information, allowing us to group data by the day of the week later.

In [27]:
orders["day"] = orders["order_date"].dt.day_name()
display(orders)


,order_id,customer_id,order_date,item_id,quantity,item_name,item_category,day
0,1,1,2020-06-01,1,10,LC Alg. Book,Book,Monday
1,2,1,2020-06-08,2,10,LC DB. Book,Book,Monday
2,3,2,2020-06-02,1,5,LC Alg. Book,Book,Tuesday
3,4,3,2020-06-03,3,5,LC SmarthPhone,Phone,Wednesday
4,5,4,2020-06-04,4,1,LC Phone 2020,Phone,Thursday
5,6,4,2020-06-05,5,5,LC SmartGlass,Glasses,Friday
6,7,5,2020-06-05,1,10,LC Alg. Book,Book,Friday
7,8,5,2020-06-14,4,5,LC Phone 2020,Phone,Sunday
8,9,5,2020-06-21,3,5,LC SmarthPhone,Phone,Sunday


**Step 3. Get Unique Item Categories**
- Here, the unique values from the item_category column in the items DataFrame are extracted, sorted, and converted into a list. This list ensures that all categories are represented in the final analysis, even if some categories do not appear in the orders data.

In [28]:
unique_categories = sorted(items["item_category"].unique().tolist())
print(unique_categories)

['Book', 'Glasses', 'Phone', 'T-shirt']


**Step 4. Set Item Category as an Ordered Categorical Variable**
- The item_category column in the orders DataFrame is converted into a categorical data type. The set_categories method ensures that the categories are explicitly ordered based on the unique_categories list. This step ensures consistency in the category ordering during the analysis.

In [29]:
orders["item_category"] = orders["item_category"].astype("category").cat.set_categories(unique_categories)
display(orders)

,order_id,customer_id,order_date,item_id,quantity,item_name,item_category,day
0,1,1,2020-06-01,1,10,LC Alg. Book,Book,Monday
1,2,1,2020-06-08,2,10,LC DB. Book,Book,Monday
2,3,2,2020-06-02,1,5,LC Alg. Book,Book,Tuesday
3,4,3,2020-06-03,3,5,LC SmarthPhone,Phone,Wednesday
4,5,4,2020-06-04,4,1,LC Phone 2020,Phone,Thursday
5,6,4,2020-06-05,5,5,LC SmartGlass,Glasses,Friday
6,7,5,2020-06-05,1,10,LC Alg. Book,Book,Friday
7,8,5,2020-06-14,4,5,LC Phone 2020,Phone,Sunday
8,9,5,2020-06-21,3,5,LC SmarthPhone,Phone,Sunday


**Step 5. Set Day of the Week as an Ordered Categorical Variable**
- Similarly, the day column is converted into a categorical data type. The set_categories method sets the days of the week in chronological order. This ensures that when the data is grouped or sorted, the days will appear in the correct sequence from Monday to Sunday.

In [30]:
orders["day"] = orders["day"].astype("category").cat.set_categories(['Monday', 
                   'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
display(orders)

,order_id,customer_id,order_date,item_id,quantity,item_name,item_category,day
0,1,1,2020-06-01,1,10,LC Alg. Book,Book,Monday
1,2,1,2020-06-08,2,10,LC DB. Book,Book,Monday
2,3,2,2020-06-02,1,5,LC Alg. Book,Book,Tuesday
3,4,3,2020-06-03,3,5,LC SmarthPhone,Phone,Wednesday
4,5,4,2020-06-04,4,1,LC Phone 2020,Phone,Thursday
5,6,4,2020-06-05,5,5,LC SmartGlass,Glasses,Friday
6,7,5,2020-06-05,1,10,LC Alg. Book,Book,Friday
7,8,5,2020-06-14,4,5,LC Phone 2020,Phone,Sunday
8,9,5,2020-06-21,3,5,LC SmarthPhone,Phone,Sunday


**Step 6. Create a Pivot Table**
- A pivot table is created to summarize the data. Each row corresponds to an item_category, and each column represents a day of the week. The table calculates the total quantity of items sold for each category on each day using the sum aggregation function. The observed=False ensures that all categories and days are included, even if no data exists for them.

In [31]:
orders = pd.pivot_table(orders, 
                        index="item_category",
                        columns="day",
                        values="quantity",
                        aggfunc="sum",
                        observed=False)
display(orders)

day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
item_category,,,,,,,
Book,20,5,0,0,10,0,0
Glasses,0,0,0,0,5,0,0
Phone,0,0,5,1,0,0,10
T-shirt,0,0,0,0,0,0,0


**Step 7. Reset the Index and Rename Columns**
- The pivot table's index (which represents item_category) is reset to a regular column named category. This makes the table easier to work with and aligns with the desired output format.

In [32]:
orders = orders.reset_index(names="category")
display(orders)

day,category,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Book,20,5,0,0,10,0,0
1,Glasses,0,0,0,0,5,0,0
2,Phone,0,0,5,1,0,0,10
3,T-shirt,0,0,0,0,0,0,0


**Step 8. Sort by Category**
- Finally, the data is sorted alphabetically by the category column. This ensures the output is neatly ordered and easier to interpret.

In [33]:
orders = orders.sort_values(by=["category"])
display(orders)

day,category,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Book,20,5,0,0,10,0,0
1,Glasses,0,0,0,0,5,0,0
2,Phone,0,0,5,1,0,0,10
3,T-shirt,0,0,0,0,0,0,0


References: [1] https://leetcode.com/problems/sales-by-day-of-the-week/description/?lang=pythondata